In [ ]:
# default_exp utils

In [ ]:
%load_ext autoreload
%autoreload 2

# create references

> API details.

In [ ]:
#hide
from nbdev.showdoc import *
import random

In [ ]:
# export
import os
import sys
import re
import argparse
import ftplib
from create_reference import defaults

In [ ]:
# export
def get_args():
    parser = argparse.ArgumentParser(prog='fetchr',
                                     description='Fetch and Generate references for bioinformatics analysis',
                                     formatter_class=argparse.ArgumentDefaultsHelpFormatter
                                    )
    
    parser.add_argument('--species',nargs='+',choices=defaults.species,default=['homo_sapiens','mus_musculus'],
                       help='Reference or references from which species')
    parser.add_argument('--indexs',nargs='+',choices=defaults.softwares,default='samtools',
                       help='Using which software(s) to create indexes')
    parser.add_argument('--reference-version','-rv',default=99,type=int,
                       help='For homo_spaiens, version=75 is the last version of Grch37 reference, \
                       you can check the version in ftp://ftp.ensembl.org/pub/')
    parser.add_argument('--outdir','-o',default='./',
                       help='Reference and indexes generated direction')
    parser.add_argument('--thread','-t',default=4,
                       help='Thread number')
    return parser.parse_args()


In [ ]:
# args=get_args('-h')

In [ ]:
# export 

def file_exists(f):
    return os.path.exists(f)

In [ ]:
file_exists('./00_utils.ipynb')


True

In [ ]:
# export

def get_ffp(species):
    return re.compile(defaults.fasta_file_pattern.format(species=species),re.IGNORECASE)

def get_gfp(species,version):
    return re.compile(defaults.gtf_file_pattern.format(species=species,version=version),re.IGNORECASE)

In [ ]:
# export

def get_likely_file_from_ftp(ftp,ftp_server,version,species,ftype,dtype,pattern):
    if ftype == 'fasta':
        direction='/pub/release-{version}/{ftype}/{species}/{dtype}/'.format(
            version=version,
            species=species,
            ftype=ftype,
            dtype=dtype)
    elif ftype == 'gtf':
        direction='/pub/release-{version}/{ftype}/{species}/'.format(
            version=version,
            species=species,
            ftype=ftype)
    try:
#         print(direction)
        ftp.cwd(direction)
        files = ftp.nlst()
        for f in files:
#                 print(f)
            if len(pattern.findall(f))>0:
                return 'ftp://'+ftp_server+direction+f
        raise ValueError('No fit ' + ftype +' file in ftp://'+ftp_server+direction)
    except ftplib.all_errors as e:
        print(e)
        sys.exit(1)
        

In [ ]:
print(get_gfp('homo_sapiens',99))
with ftplib.FTP(defaults.ensembl_ftp) as ftp:
    ftp.login()
    for s in random.sample(defaults.species,2):
        print(s)
        print(get_likely_file_from_ftp(ftp,defaults.ensembl_ftp,'99',s,'fasta','dna',get_ffp(s)))
        print(get_likely_file_from_ftp(ftp,defaults.ensembl_ftp,'99',s,'gtf',None,get_gfp(s,99)))


re.compile('homo_sapiens.+?99.gtf.gz', re.IGNORECASE)
anser_cygnoides
ftp://ftp.ensembl.org/pub/release-99/fasta/anser_cygnoides/dna/Anser_cygnoides.GooseV1.0.dna.toplevel.fa.gz
ftp://ftp.ensembl.org/pub/release-99/gtf/anser_cygnoides/Anser_cygnoides.GooseV1.0.99.gtf.gz
pongo_abelii
ftp://ftp.ensembl.org/pub/release-99/fasta/pongo_abelii/dna/Pongo_abelii.PPYG2.dna.toplevel.fa.gz


In [ ]:
# export

def get_local_files(outdir,species,version):
    sample_outdir='{outdir}/{species}/{version}'.format(outdir=outdir,species=species,version=version)
    local_genome_fasta='{sample_outdir}/genome.fa.gz'.format(sample_outdir=sample_outdir)
    local_transcriptome_gtf = '{sample_outdir}/transcriptome.gtf.gz'.format(sample_outdir=sample_outdir)
    os.makedirs(sample_outdir,exist_ok=True)
    return local_genome_fasta,local_transcriptome_gtf

In [ ]:
# export

def get_paras(args,ftp,ftp_server):
    paras=[]
    for sp in args.species:
        para={}
        para['species']=sp
        para['link_genome_fasta']=get_likely_file_from_ftp(ftp,
                                                           ftp_server,
                                                           args.reference_version,
                                                           sp,
                                                           'fasta',
                                                           'dna',
                                                           get_ffp(sp))
        para['link_transcriptome_gtf']=get_likely_file_from_ftp(ftp,
                                                                ftp_server,
                                                                args.reference_version,
                                                                sp,'gtf',
                                                                None,
                                                                get_gfp(sp,
                                                                        args.reference_version))
        para['local_genome_fasta'],para['local_transcriptome_gtf'] = get_local_files(args.outdir,
                                                                                     sp,
                                                                                     args.reference_version)
        paras.append(para)
        
    return paras

In [ ]:
args=get_args()

In [ ]:
with ftplib.FTP(defaults.ensembl_ftp) as ftp:
    ftp.login()
#     ftp.cwd('/pub/release-99/fasta/homo_sapiens')
#     print(ftp.nlst())
#     ftp.cwd('/pub/release-99/fasta/homo_spaiens/dna/')
    print(get_paras(args,ftp,defaults.ensembl_ftp))

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()